In [27]:
INPUTFILE = 'input_5.txt'

with open(INPUTFILE, 'r') as file:
    content = file.read()
rows = (content.split("\n"))[:-1]

Part 1

In [28]:
packs = content.split("\n\n")
seeds = [int(seed) for seed in packs[0].split(":")[1].split(" ") if seed]

for pack in packs[1:]:
    maps = [x for x in pack.split("\n") if x]
    
    seed_changed = [False for _ in seeds]
    for current_map in maps[1:]:
        map_numbers = current_map.split(" ")
        dest_range_start = int(map_numbers[0])
        source_range_start = int(map_numbers[1])
        range_length = int(map_numbers[2])
        for seed_index in range(len(seeds)):            
            if seeds[seed_index] in range(source_range_start, source_range_start + range_length) and not seed_changed[seed_index]:
                seeds[seed_index] = dest_range_start + seeds[seed_index] - source_range_start
                seed_changed[seed_index] = True
print(min(seeds))

51580674


Part 2

In [29]:
packs = content.split("\n\n")
seeds = [int(seed) for seed in packs[0].split(":")[1].split(" ") if seed]

maps = [[list(map(int, x.split(" "))) for x in pack.split("\n")[1:] if x] for pack in packs[1:]]
for mapping in maps:
  mapping.sort(key=lambda x: x[1])

# for seed_index in range(0, len(seeds), 2):
#     start = seeds[seed_index]
#     end = start + seeds[seed_index + 1]
#     for current_map in maps:
#         for map_rule in current_map:
#             if start > 

In [30]:
def lowest_range_location(seeds: list[int], maps: list) -> int:
  lowest_locations = []

  for start, length in zip(*[iter(seeds)] * 2):
    src_ranges = [(start, start + length - 1)]
    dst_ranges = []

    for mapping in maps:
      for lo, hi in src_ranges:
        for dest, src, dist in mapping:

          # entire source range is less than any mapping
          # pass through source range as destination range
          if hi < src:
            dst_ranges.append((lo, hi))
            break

          # source range is contained within this range
          elif lo >= src and lo < src + dist:

            # the end of the source range is past this range
            # add a destination range for the part within this range
            # and continue processing the rest of the source range
            if hi >= src + dist:
              dst_ranges.append((dest + (lo - src), dest + dist - 1))
              lo = src + dist

            # the source range is wholly contained within this range
            # map the entire range as the destination range
            else:
              dst_ranges.append((dest + (lo - src), dest + (hi - src)))
              break

          # part of the source range is less than any mapping
          # add a destination range passing through the part not in a mapping
          # and continue processing for the rest of the source range
          elif lo < src and hi >= src:
            dst_ranges.append((lo, src - 1))
            lo = src

        # the source range was not contained within any mapping
        # pass through the source range as the destination range
        else:
          dst_ranges.append((lo, hi))

      # results of one mapping become the ranges for the next
      src_ranges = dst_ranges
      dst_ranges = []

    # find the lowest result out of all the final ranges
    lowest_locations.append(min(lo for lo, _ in src_ranges))

  return min(lowest_locations)

In [31]:
print(lowest_range_location(seeds, maps))

105230362


In [5]:
import numpy as np
from tqdm import tqdm
from functools import reduce

In [16]:
def convert_seed_through_all_maps(seed: int) -> int:
    def apply_current_map(current_seed, map_rules):
        conditions = np.logical_and(current_seed >= map_rules[:, 1], current_seed < map_rules[:, 1] + map_rules[:, 2])
        index = np.argmax(conditions)
        return map_rules[index, 0] + current_seed - map_rules[index, 1] if np.any(conditions) else current_seed

    return reduce(apply_current_map, maps, seed)

packs = content.split("\n\n")
seeds = [int(seed) for seed in packs[0].split(":")[1].split(" ") if seed]

global maps
maps = []
lowest_seed = np.inf

for pack in packs[1:]:
    map_pack = [x.split(" ") for x in pack.split("\n") if x]
    maps.append(np.array(map_pack[1:], dtype=np.int64))

for seed_index in range(0, len(seeds), 2):
    start = seeds[seed_index]
    end = start + seeds[seed_index + 1]
    current_seeds = np.arange(start, end)
    new_seeds = np.vectorize(convert_seed_through_all_maps)(current_seeds)
    lowest_seed = min(lowest_seed, np.min(new_seeds))

print(lowest_seed)


46


In [15]:
import numpy as np

def convert_seed_through_all_maps(seed: int, maps):
    current_seed = seed
    for map_rules in maps:
        conditions = (current_seed >= map_rules[:, 1]) & (current_seed < map_rules[:, 1] + map_rules[:, 2])
        if np.any(conditions):
            index = np.argmax(conditions)
            current_seed = map_rules[index, 0] + current_seed - map_rules[index, 1]
    return current_seed

packs = content.split("\n\n")
seeds = [int(seed) for seed in packs[0].split(":")[1].split(" ") if seed]

maps = [np.array([list(map(int, x.split(" "))) for x in pack.split("\n")[1:] if x], dtype=np.int64) for pack in packs[1:]]

lowest_seed = np.inf

for seed_index in range(0, len(seeds), 2):
    start = seeds[seed_index]
    end = start + seeds[seed_index + 1]

    for current_seed in range(start, end):
        current_seed = convert_seed_through_all_maps(current_seed, maps)
        lowest_seed = min(lowest_seed, current_seed)

print(lowest_seed)


46
